In [70]:
import pandas, numpy, os, re

In [71]:
article_directory = './hw01-data-raw/News Articles'
article_list = os.listdir(article_directory) # create list of files
article_list[:5]

['1101243072871.txt',
 '1101163341619.txt',
 '1101163941020.txt',
 '1101163268260.txt',
 '1101163374885.txt']

In [72]:
keywords = { 
    'mad cow': ['mad cow', 'prion', 'creutzfeldt-jakob', 'spongiform', 'encephalopath'], 
    'laboratory': ['lab'],
    'whistleblower': ['whistle'],
    'key names': ['Ryker', 'Sanderson', 'Boynton', 'Dortmund', 'Sulfate'],
    'Laurie/Laurel Sulfate': ['Sulfate', 'Laurie', 'Laurel'],
    'animal feed': ['feed'],
    'cash flow': ['$', 'dollar', 'money', 'cash', 'expense'],
    'mayor': ['locke']
}
keywords.keys()

dict_keys(['mad cow', 'laboratory', 'whistleblower', 'key names', 'Laurie/Laurel Sulfate', 'animal feed', 'cash flow', 'mayor'])

regular expression from https://stackoverflow.com/questions/17268958/finding-occurrences-of-a-word-in-a-string-in-python-3

In [73]:
article_data = []
# create dataframe from list of dictionaries
# each dictionary: {filename: filename.txt, title: "", author: {author name}, date: {date posted}, ...}

for file_name in article_list:
    # open the file
    article = open(f'{article_directory}/{file_name}', encoding='ISO 8859-1')
    article_text = article.read()
    # replace weird characters and newlines with single space, convert to lower case
    # article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', ' ')
    # article_text = re.sub(' +', ' ', article_text).lower()

    # empty list of article keywords to populate
    article_keywords = []
    article_dict = { 'filename': file_name}
    # get other metadata for article_dict from Emi's code (article title, author, date published)
    article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', '#')
    article_text = re.sub(' +', ' ', article_text).lower()
    article_text = re.sub('(#)+', '#', article_text)
    article_text = article_text[1:]
    article_pieces =  article_text.split('#')
    # print(article_pieces)
    title = article_pieces[0]
    author = False
    date = False
    if "story by:" in article_pieces[1]:
        author = article_pieces[1][10:]
        date = article_pieces[2][23:]
    elif article_pieces[0] == " ":
        date = article_pieces[1].strip()[23:]
    else:
        date = article_pieces[2].strip()[23:]
    article_dict['title'] = title
    article_dict['date'] = date
    article_dict['author'] = author
    # tally up occurrences of each keyword category
    tally = 0
    for keyword in keywords.keys():
        article_dict[keyword] = 0
        contains_keyword = False
        for synonym in keywords[keyword]:
            num_found = sum(1 for _ in re.finditer(r'%s' % re.escape(synonym.lower()), article_text))
            article_dict[keyword] += num_found
            tally+=num_found
    if tally>0:
        article_data.append(article_dict)

In [74]:
article_df = pandas.DataFrame(article_data)
article_df['date'] = pandas.to_datetime(article_df['date'], format='mixed')
# article_df.sort_values(by='key names', ascending=False)
param = 'mayor'
article_df[article_df[param] > 0].sort_values(by='date')

,filename,title,date,author,mad cow,laboratory,whistleblower,key names,Laurie/Laurel Sulfate,animal feed,cash flow,mayor
21,1101243270826.txt,briefs,2002-02-17,False,0,0,0,0,0,0,1,1
448,1101243249513.txt,blanket primary eliminated in washington state,2002-02-24,False,0,0,0,0,0,0,0,1
334,1101163559104.txt,"dollar tree open, sales are booming",2002-03-09,jamie smart,0,0,0,0,0,0,5,1
250,1101163478729.txt,spady family epitomizes america's citizen acti...,2002-03-24,False,0,0,0,0,0,0,1,2
402,1101163356450.txt,laboratory groundbreaking,2002-05-05,john panni,1,2,0,3,0,0,1,2
507,1101163306213.txt,open mike,2002-05-19,john panni,0,0,0,0,0,0,1,1
29,1101163251635.txt,mabton girls coach taking his game to a differ...,2002-06-04,john panni,0,0,0,0,0,0,0,1
543,1101163161541.txt,forum,2002-06-30,False,0,0,0,0,0,0,5,2
271,1101163852951.txt,gov. locke treks to lower valley to allay mad ...,2003-01-15,jamie smart,5,1,0,0,0,1,0,11
13,1101163777028.txt,elerding sets state qualifying times,2003-01-28,False,0,0,0,0,0,0,0,1
